<a href="https://colab.research.google.com/github/xx529/Algorithm/blob/main/Tobit%20Model%20-%20%20Censored%20Normal%20Regression/Tobit%20Model%20-%20%20Censored%20Normal%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tobit Model -  Censored Normal Regression

**关于算法的故事**

在任一给定年份，有相当数量家庭的医疗保险费用支出为0

因此，虽然年度家庭医疗保险费用支出的总体分布散布于一个很大的正数范围内，但在数字0上却相当集中

# Algorithm

**定义**

1. 因变量 $y$ 在正值上大致是连续分布，但也包含一部分以正概率取值为0的观测，对于 $y$ 如下定义，$y^*$ 是一个隐藏的变量，它与0共同决定了观测 $y$

$$y=\left\{
\begin{aligned}
y^* ,& y^* \geqslant 0 \\
- ,& y^* < 0 
\end{aligned}
\right.
$$

2. 隐藏变量 $y^*$ 由一个参数为 $\beta$ 的线性方程和 $\epsilon$，定义如下
2
$$y^* = x^T\beta + \epsilon, \text{ 其中 } \epsilon \sim N(0, \sigma^2)$$
$$$$

3. 需要优化的参数就是 $\beta$ 和 $\sigma$

$$\hat{\theta}= (\hat{\beta}, \hat{\sigma}^2)$$

**目标函数**

1. 定义一个 indicator variable $d$，取 $L=0$ 如下:

$$d=\left\{
\begin{aligned}
1 ,& y > L\\
0 ,& y = L 
\end{aligned}
\right.$$

2. 由定义式可知 $y^* \sim (x^T\beta, \sigma^2)$

$$y^* = x^T\beta + \epsilon \rightarrow y^* \sim (x^T\beta, \sigma^2)$$

3. 当 $y^*<L$，用 CDF概率分布函数，当 $y^*>L$ 可观测时候用 PDF概率密度函数，这里取 $L=0$

$$f(y)=\left\{
\begin{aligned} F^*(0)=1-\Phi(\frac{x^T\beta}{\sigma}), d=0\\
\frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{1}{2\sigma^2}(y-x^T\beta)^2), d=1
\end{aligned}
\right.
$$

4. 综合上面两个分段函数就有如下形式，也就是我们通过 MLE 最大化的函数

$$f(y) =[\frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{1}{2\sigma^2}(y-x^T\beta)^2)]^d[1-\Phi(\frac{x^T\beta}{\sigma})]^{1-d}$$

**参数梯度**

对以上公式取 $log$ 之后的参数梯度，其中 $\phi_i=\phi(\frac{x_i^T\beta}{\sigma})$，$\Phi_i=\Phi(\frac{x_i^T\beta}{\sigma})$

$$\frac{\partial ln L_N}{\partial \beta}=\sum^N_{i=1}\frac{1}{\sigma^2}(d_i(y_i-x^T\beta)-(1-d_i)\frac{\sigma\phi_i}{1-\Phi_i})x_i$$


$$\frac{\partial ln L_N}{\partial \sigma^2}=\sum^N_{i=1}[d_i(-\frac{1}{2\sigma^2}+\frac{(y_i-x^T\beta)^2}{2\sigma^4})+(1-d_i)\frac{\phi_ix_i^T\beta}{1-\Phi_i} · \frac{1}{2\sigma^3}]$$

# Implementation

In [11]:
import jax
import jax.numpy as jnp
from jax import grad, vmap
import numpy as np

## Data

In [12]:
key = jax.random.PRNGKey(199)
batch = 10000
input_dim = 5
L = 0

true_beta = jnp.array([2.0, 1.0, -2.0, 3.0, 4.0])
true_sigma = 2.0
epsilon = jax.random.normal(key, (batch, )) * (true_sigma**2)

x = jax.random.normal(key, (batch, input_dim))
y_star = jnp.dot(x, true_beta) + epsilon
d = (y_star > L).astype(jnp.float32)
y =  d * y_star

beta = jax.random.normal(key, (input_dim, ))
sigma = jax.random.normal(key)

print('epsilon mean', epsilon.mean())
print('epsilon std', epsilon.std())
print('y_star std', y_star.std())
print('x_dot_b mean', jnp.dot(x, true_beta).mean())
print('y_star mean', y_star.mean())

epsilon mean 0.0054394654
epsilon std 3.9888053
y_star std 7.04811
x_dot_b mean -0.012680665
y_star mean -0.0072411946


In [13]:
y

DeviceArray([-0.        ,  5.2114363 ,  9.080499  , ...,  0.13754821,
              0.43961388, -0.        ], dtype=float32)

In [14]:
y_star

DeviceArray([-5.634598  ,  5.2114363 ,  9.080499  , ...,  0.13754821,
              0.43961388, -5.3524485 ], dtype=float32)

## Function

In [15]:
def normal_cdf(x):
    return jax.scipy.stats.norm.cdf(x, loc=0, scale=1)

def normal_pdf(x):
    return jax.scipy.stats.norm.pdf(x, loc=0, scale=1)

def linear(x, beta):
    return jnp.dot(x, beta)

def log_mle(x, beta, sigma, y_true):
    d = (y_star > L).astype(jnp.float32)
    temp1 = d * jnp.log(1 / (jnp.sqrt(2 * jnp.pi) * sigma)) + (-1 / (2 * sigma**2)) * (y_true - linear(x, beta))**2
    temp2 = (1 - d) * jnp.log(1 - normal_cdf(linear(x, beta) / sigma) + 0.001)
    return jnp.mean(temp1 + temp2)

def get_beta_grad(x, y_true, beta, sigma):
    pass

def get_sigma2_grad(x, y_true, beta, sigma):
    pass

def tobit_model_grad(x, y_true, beta, sigma):
    x_dot_beta = linear(x, beta)
    residual = y_true - x_dot_beta
    sigma_square = sigma**2
    cdf = normal_cdf(x_dot_beta)
    pdf = normal_pdf(x_dot_beta)
    d = (y_star > L).astype(jnp.float32)
    
    d_beta_temp = d*residual - (1-d) * sigma * pdf / (1 - cdf + 0.01)
    d_beta = - (1 / sigma_square) * jnp.dot(d_beta_temp, x) / x.shape[0]
    
    d_sigma2_temp_1 = d * (-1 / 2 * sigma_square + (residual**2) / (2 * sigma**4))    
    d_sigma2_temp_2 = 1 / (2 * sigma**3 ) * (1 - d) * (pdf * x_dot_beta) / (1 - cdf + 0.01)
    d_sigma2 = - jnp.mean(d_sigma2_temp_1 + d_sigma2_temp_2)
    
    return d_beta, d_sigma2
    

def tobit_model_train(x, y_true, beta, sigma, lr, esp, max_iter):
    current_mle = jnp.inf
    current_esp = jnp.inf
    current_iter = 1

    while current_iter < max_iter and current_esp > esp:
        d_beta, d_sigma2 = tobit_model_grad(x, y, beta, sigma)

        beta = beta - lr * d_beta
        sigma = jnp.sqrt(sigma**2 - lr * d_sigma2)

        mle = log_mle(x, beta, sigma, y_true)
        # current_esp = jnp.abs(mle - current_mle)
        # current_mle = mle
        current_iter += 1

        if current_iter % 100 == 0:
            print(beta, sigma, mle)

    return beta, sigma

In [16]:
lr = 1e-3
esp = 1e-9
max_iter = 10000

opt_beta, opt_sigma = tobit_model_train(x, y, beta, sigma, lr, esp, max_iter)

[ 3.0073717   0.8210611  -1.6421138  -0.950164    0.67173666] 1.3290615 -9.184137
[ 2.9784222   0.82507724 -1.6523643  -0.8457782   0.76444256] 1.41895 -7.9925284
[ 2.9525461   0.828536   -1.6611848  -0.7538167   0.84594196] 1.4803617 -7.2721853
[ 2.9286487  0.8316117 -1.6690612 -0.6699635  0.9200998] 1.5260867 -6.772764
[ 2.9061747   0.8343986  -1.676245   -0.59200746  0.98890275] 1.5617735 -6.3992405
[ 2.8847916   0.83695513 -1.6828912  -0.5186228   1.0535463 ] 1.5904524 -6.106111
[ 2.8642836   0.8393223  -1.6891018  -0.44893494  1.114818  ] 1.6139499 -5.8683395
[ 2.8445034   0.84153193 -1.6949465  -0.38233507  1.1732696 ] 1.6334597 -5.6707945
[ 2.8253496   0.84360874 -1.7004765  -0.31838137  1.2293006 ] 1.6498058 -5.503695
[ 2.806748   0.8455712 -1.7057322 -0.2567395  1.283213 ] 1.6635854 -5.3603745
[ 2.7886417  0.8474326 -1.7107447 -0.1971442  1.3352443] 1.6752436 -5.2361035
[ 2.7709837   0.8492029  -1.7155393  -0.13938063  1.3855867 ] 1.6851224 -5.1274066
[ 2.753737    0.85089123 

In [17]:
print(opt_beta)
print(true_beta)

[ 2.0448775   0.94659066 -1.9302727   2.2694583   3.4781184 ]
[ 2.  1. -2.  3.  4.]


In [18]:
print(opt_sigma)
print(true_sigma)

1.6107825
2.0
